In [1]:
from Cryptodome.Hash import SHA256
from Cryptodome.PublicKey import ECC
from Cryptodome.Signature import DSS
from Cryptodome.Random import get_random_bytes
from Cryptodome.Cipher import AES, PKCS1_OAEP
import secrets

from BlockchainFunctions import *

In [2]:
class User():
    def __init__(self,name):
        """
        Constructor for the User class
        
        input: [name], a string that represents the name of the user
        
        output: [self.publicKey], an object of type ECC.publicKey that represents the coordinates for the 
        users public key
        
        output: [self.privateKey], an object of type ECC.privateKey that represents the users private key. 
        The private key can be obtained by using the following methods: privateKey.d
        """
        ##Calling the generateKeys method of the User class to set the keys
        self.publicKey, self.privateKey = User.generateKeys()
        self.name = name
        
    def generateKeys():
        """
        Generates a set of public and private keys 
        
        output: [public], an object of type ECC.publicKey that represents the coordinates for the 
        users public key
        
        output: [private], an object of type ECC.privateKey that represents the users private key. 
        The private key can be obtained by using the following method: privateKey.d
        """
        
        #Generating the private key using the P-256 curve
        private = ECC.generate(curve = 'P-256')
        #Using the private key to generate the public key. This is done by doing privateKey * generator point
        public = private.public_key()
        return public,private
    
    def sendMessage(self, reciever, message, blockchain):
        """
        Encrypts and digitally signs the mesage before adding it to the blockchain
        
        input: [self], an object of the User class that represents the sender
        
        input: [reciever], an object of the User class that represents the receiver
        
        input: [message], a string that represents the message to be sent
        
        input: [blockchain], an object of the Blockchain class that the new block will be added to 
        
        output: [(self.digitallySignMessage(encryptedMessage)], the output represents the value that is
        returned by the digitallySignMessage() method of the User class
        """
        #Checks if the message is correctly encoded. If not, the message is encoded using UTF-8 encoding. 
        if not isinstance (message, bytes):
            message = message.encode('UTF-8')
        #Collects the recievers public Key
        publicKey = reciever.publicKey
        #Converts the users public key into a point on the Elliptc Curve
        publicKeyPoint = ECC.EccPoint(publicKey.pointQ.x, publicKey.pointQ.y, curve='p256')
        #Creates a new, one time private Key
        oneTimePrivateKey = ECC.generate(curve = 'P-256')
        #The key that is used for encryption is the scalar multiplication of the the oneTimePrivate key and
        #the recievers public key
        sharedECCKey = int(oneTimePrivateKey.d) * publicKeyPoint
        #The shared key is hashed so it can be used for AES encryption
        secretKey = User.pointToKey(sharedECCKey)
        #AES encryption is performed ont the message using the hashed key
        ciphertext, nonce, authTag = User.AESEncryption(message, secretKey)
        #The public key of the one time private key is generated
        oneTimePublicKey = oneTimePrivateKey.public_key()
        #The encrypted message comprises the result of the encryption, as well as the one time public key
        encryptedMessage = (ciphertext, nonce, authTag, oneTimePublicKey)
        #The new block is added to the blockchain
        blockchain.addBlock(ciphertext)
        #The encrypted message is digitally signed by the sender. The output is returned
        return (self.digitallySignMessage(encryptedMessage))
    
    def recieveMessage(self, sender, message):
        """
        If the digital signature is valid, the message is decrypted
        
        input: [self], an object of the User class that represents the reciever of the message
        
        input: [sender], an object of the User class that represents the sender of the message
        
        input: [message], a tuple that has has the encrypted message data and teh digital signature
        
        output: [plaintext], a bytes string that represents the decrypted message
        """
        #collects the receivers private key
        privateKey = self.privateKey
        # seperates the message data into the data from encryption (ciphertext, none, authTag) and the 
        #oneTimePublicKey
        ciphertext, nonce, authTag, oneTimePublicKey = message
        #Collects the privateKey value from the ECCKey object
        privateKeyD = int(privateKey.d)
        #Converts the public key into a point from an ECCKey object
        PublicKeyPoint = ECC.EccPoint(oneTimePublicKey.pointQ.x, oneTimePublicKey.pointQ.y, curve='p256')
        #Generates the sharedECCKey from the recievers private key and the shared publicKey
        sharedECCKey = privateKeyD * PublicKeyPoint
        #Hashes the shared key so it can be used for decryption
        secretKey = User.pointToKey(sharedECCKey)
        #Comprises the plaintext from the decrypted message
        plaintext = User.AESDecryption(ciphertext, nonce, authTag, secretKey)
        #Returns the decrypted message
        print (f"Recieved Message: {plaintext.decode('utf-8')}")
        return plaintext

    def digitallySignMessage(self, encryptedData):
        """
        Digitally signs the encrypted message
        
        input: [self], an object of the User class that represents the sender
        
        input: [encryptedData] a tuple created by the sendMessage method of the User class thar is of the 
        following format (ciphertext, nonce, authTag, oneTimePublicKey)
        
        output: [encryptedData], returns the encryptedData unchanged
        
        output: [signature], the digital signature of the encryptedData
        """
        #Collects the senders private key
        privateKey = self.privateKey
        #Collects the encrypted data
        message = encryptedData
        #Converts the first 3 elements of the encrypted data into a bytes string. Doesn't include the ECCKey
        #object. 
        messageString = b''.join(message[:-1])
        #Adds the x and y coordinates of the public key to the string
        messageString = messageString + bytes((str(message[-1].pointQ.x) + str(message[-1].pointQ.y)).encode('UTF-8'))
        #Hashing the string using SHA256
        h = SHA256.new(messageString)
        #Setting up the signer using the FIPS-186-3 protocol
        signer = DSS.new(privateKey, 'fips-186-3')
        #Digitally signing the hashed data
        signature = signer.sign(h)
        #Returns the encryptedData and the Digital signature
        return encryptedData, signature
    
    def checkSignedMessage(self, sender, encryptedData):
        """
        Checks if the digital signature is valid
        
        input: [self], an object of the user class that represents the reciever
        
        input: [sender], an object of the user class that represents the sender
        
        input: [encryptedData], a tuple that comprises the encrypted message and the signed data. 
        
        output: [self.recieveMessage(sender, message)], the output of the recieveMessage() method of the User
        class
        """
        
        #Seperating the message and digital signature from the encryptedData input
        message, signature = encryptedData
        #Converts the first 3 elements of the encrypted data into a bytes string. Doesn't include the ECCKey
        #object.  
        messageString = b''.join(message[:-1])
        #Adds the x and y coordinates of the public key to the string
        messageString = messageString + bytes((str(message[-1].pointQ.x) + str(message[-1].pointQ.y)).encode('UTF-8'))
        #Hashing the string using SHA256
        h = SHA256.new(messageString)
        #Setting up the verifier using the FIPS-186-3 protocol
        verifier = DSS.new(sender.publicKey, 'fips-186-3')
        
        #Tries to verify the encrypted message using the verifier. If the verification fails, a ValueError
        #is raised
        try:
            #Attempting to verify the valdity of the signature
            verifier.verify(h, signature)
            print('Message has been verified')
            #If the verification is successful, the recieveMessage function is calles to decrypt the data
            return self.recieveMessage(sender, message)
        except ValueError:
            print ("digital signature doesn't match")
            
    def AESEncryption(message, secretKey):
        """
        Encrypting the message using AES Encryption
        
        input: [message], a bytes string that represents the message to be encrypted
        
        input: [secretKey], a 256 bit string that is used for the encryption
        
        output: [(ciphertext, aesCipher.nonce, authTag)], a tuple comprised of bytes strings that 
        represents the encrypted message
        
        """
        #Setting up the encrypter using the secret key
        aesCipher = AES.new(secretKey, AES.MODE_GCM)
        #Encrypting the message
        ciphertext, authTag = aesCipher.encrypt_and_digest(message)
        return (ciphertext, aesCipher.nonce, authTag)

    def AESDecryption(ciphertext, nonce, authTag, secretKey):
        """
        Decrypting the message using AES Decryption
        
        input: [(ciphertext, nonce, authTag)], The outputs of the AESEncryption() method of the User class 
        that represent the encrypted message
        
        input: [secretKey], a 256 bit bytes string that represents the key used for encryption
        
        output: [plaintext], a bytes string representing the decrypted message
        """
        aesCipher = AES.new(secretKey, AES.MODE_GCM, nonce)
        plaintext = aesCipher.decrypt_and_verify(ciphertext, authTag)
        return plaintext

    def pointToKey(point):
        """
        Converting a point into a 256 bit key
        
        input: [point], an EccPoint object that represents the key to be used for encryption
        
        output: [h.digest()], a bytes string representing the SHA256 bit hash of the coordinates of the point
        
        """
        #Converting the x coordinate to a bytes string and setting up the SHA256 hasher
        h = SHA256.new(int.to_bytes(int(point.x), 32, 'big'))
        #Updating the hasher with the y coordinate of the point as a bytes string
        h.update(int.to_bytes(int(point.y), 32, 'big'))
        #Returning the hash in base64 encoding
        return h.digest()

In [3]:
blockchain = Blockchain()
    
def test():
    """
    Creates 2 new users, 1 sender and 1 reciever. Takes a string as an input as the message to be sent. 
    
    Encrypts the mesage using the recievers public key. The encryption function also returns the digital 
    signature along with the encrypted message. The digital signature is created using the senders
    private key. 
    
    The reciever checks the signature and encrypted message using the senders public key. If it is a valid 
    signature, the reciever decrypts the message. 
    """
    
    sender = User(input('Please enter the name of the sender: '))
    reciever = User(input('Please enter the name of the reciever: '))
    
    message = input(f'{sender.name}, please type in your message: ')
    
    encryptedMessage = sender.sendMessage(reciever, message, blockchain)
    
    print(f'The message has been added to the blockchain! The length of the chain is now {len(blockchain.chain)}')
    
    reciever.checkSignedMessage(sender, encryptedMessage)

In [4]:
test()

Please enter the name of the sender: User 1
Please enter the name of the reciever: User 2
User 1, please type in your message: Hello, this is a test message
The message has been added to the blockchain! The length of the chain is now 2
Message has been verified
Recieved Message: Hello, this is a test message
